In [1]:
import requests

In [ ]:
ddir = "./results"

## 复旦专业目录

In [5]:
# Fudan 专业目录 https://gsas.fudan.edu.cn/sszsml2022/index.html 
data_url = "https://gsas.fudan.edu.cn/sszsml2022/js/data.js?v=1636982382000"
s = requests.session()
data = s.get(data_url).content

In [7]:
data.decode('utf-8')[:100]

'var treeData = {"text":"复旦大学","value":"0","items":[{"text":"001 马克思主义学院","value":"001","YXSH":"001",'

In [12]:
# parts = data.decode('utf-8').split("=")
# parts

In [13]:
with open(f'{ddir}/fudan-major.js', 'w') as f:
    f.write(data.decode('utf-8'))

In [20]:
import json
tree_data = json.load(open(f'{ddir}/tree-data.json', 'r'))
main_data = json.load(open(f'{ddir}/main-data.json', 'r'))

In [21]:
tree_data.keys(), main_data.keys()

(dict_keys(['mllb_12']), dict_keys(['text', 'value', 'items']))

等级

- YXSH 学院代码？

tree 表

- 学院级别；
  - yxmc 学院名         古籍整理研究所（中国古代文学研究中心）
  - yxsh 院系代码        010
  - ZsmlYxZys 专业      
    - zymc 专业名       中国语言文学
    - zydm 专业代码      050100
    - yxsh 所属院系      010
    - ZsmlYxZyYjfx 研究方向
      - fxmc 方向名称   汉语言文字学
      - yxsh 院系       010
      - zydm 专业代码   050100

In [25]:
yx_dict = {}
zy_dict = {}

for yx in tree_data['mllb_12']:
    yx_code = yx['yxsh']
    yx_name = yx['yxmc']
    yx_tel = yx['tel']
    yx_enroll_number = yx['zszrs']  # 招生总人数
    yx_dict[yx_code] = {
        "name": yx_name,
        "code": yx_code,
        "tel": yx_tel,
        "number": yx_enroll_number
    }
    for zy in yx["exportZsmlYxZys"]:
        zy_code = zy["zydm"]
        zy_name = zy['zymc']
        zy_belong = zy['yxsh']
        zy_enroll_number = zy['zsrs']
        zy_fx = []      # 专业方向
        for fx in zy['exportZsmlYxZyYjfxs']:
            zy_fx.append([fx['fxmc'], fx['xzmc']])
        zy_dict[zy_name] = {
            "name": zy_name,
            "code": zy_code,
            "number": zy_enroll_number,
            "yx": yx_dict[zy_belong]['name'],
            "fx": zy_fx
        }

zy_dict['语言学及应用语言学']

{'name': '语言学及应用语言学',
 'code': '050102',
 'number': '10',
 'yx': '中国语言文学系',
 'fx': [['理论语言学', '3年'],
  ['文化语言学', '3年'],
  ['应用语言学', '3年'],
  ['社会语言及方言学', '3年'],
  ['言语听力科学', '3年'],
  ['对外汉语教学', '3年']]}

In [28]:
json.dump(zy_dict, open(f"{ddir}/fudan_majors.json", 'w'), ensure_ascii=False)

## QS排名地址

In [29]:
# https://www.qschina.cn/university-rankings/world-university-rankings/2022
data_url = "https://www.qschina.cn/sites/default/files/qs-rankings-data/cn/2122636_indicators.txt"
s = requests.session()
data = s.get(data_url).content

In [45]:
with open(f'{ddir}/qs-2020.json', 'w') as f:
    f.write(data.decode('unicode-escape'))      # 会把 "ind_76": "<div class=\"td-wrap\"><div class=\"td-wrap-in\">100<\/div><\/div>", 里面的 \ 删去

<ipython-input-45-3eda383fbef0>:2: DeprecationWarning: invalid escape sequence '\/'
  f.write(data.decode('unicode-escape'))


In [43]:
b'\u675c\u4f0a\u65af\u5821-\u57c3\u68ee\u5927\u5b66'.decode('unicode-escape')

'杜伊斯堡-埃森大学'

In [49]:
with open(f'{ddir}/qs-2020-raw.json', 'w') as f:
    f.write(data.decode())      # 保存为raw

In [84]:
data_list = json.loads(data.decode())["data"]

### 测试 RT

In [61]:
uni = "<div class=\"td-wrap\"><div class=\"td-wrap-in\"><a href=\"\/universities\/chung-yuan-christian-university\" class=\"uni-link\">\u4e2d\u539f\u57fa\u7763\u6559\u5927\u5b66<\/a><\/div><\/div>"
uni

'<div class="td-wrap"><div class="td-wrap-in"><a href="\\/universities\\/chung-yuan-christian-university" class="uni-link">中原基督教大学<\\/a><\\/div><\\/div>'

In [60]:
uni.replace("\\", "")

'<div class="td-wrap"><div class="td-wrap-in"><a href="/universities/chung-yuan-christian-university" class="uni-link">中原基督教大学</a></div></div>'

In [81]:
import xml.etree.ElementTree as ET
tree = ET.fromstring(uni.replace("\\", ""))

In [82]:
tree[0][0].tag, tree[0][0].text, tree[0][0].attrib

('a',
 '中原基督教大学',
 {'href': '/universities/chung-yuan-christian-university',
  'class': 'uni-link'})

### 抽取

In [83]:
import xml.etree.ElementTree as ET
def ext_name(uni_str):
    root = ET.fromstring(uni_str.replace("\\", ""))
    a = root[0][0]
    name = a.text
    url = a.attrib["href"].split("/")[-1]
    return name, url


In [86]:
qs_dict = {}
for university in data_list:
    u_rank = int(university['overall_rank'])
    u_name, u_url = ext_name(university["uni"])
    qs_dict[u_name] = {
        "name": u_name,
        "rank": u_rank,
        "url": u_url, 
        "region": university['region'],
        "location": university['location'],
    }
qs_dict["蒂宾根大学"]

{'name': '蒂宾根大学',
 'rank': 178,
 'url': 'eberhard-karls-universitat-tubingen',
 'region': 'Europe',
 'location': 'Germany'}

In [87]:
json.dump(qs_dict, open(f"{ddir}/qs-2022.json", 'w'), ensure_ascii=False)